# 第3章　scikit-learn机器学习分类器

本章将介绍学术界和行业中常用且强大的一系列机器学习算法。了解几个监督学习分类算法之间的差异，培养鉴别算法优劣的直觉。此外，scikit-learn软件库为高效且富有成果地使用这些算法提供了友好且统一的用户界面，本章将从该软件库开始。

本章将主要涵盖下述几个方面：

- 介绍强大且常用的分类算法，如逻辑回归、支持向量机和决策树。
- scikit-learn机器学习库通过对用户友好的Python API提供各种机器学习算法，本章将通过示例对其进行解释。
- 比较线性和非线性决策边界分类器的优劣。

# 3.1　选择分类算法

每种算法都是基于某些假设的而且都有各自的特点，为特定问题选择合适的分类算法需要实践经验。David·H. Wolpert提出的天下没有免费午餐的定理，明确说明不存在适合所有可能场景的分类算法^[The Lack of A Priori Distinctions Between Learning Algorithms, Wolpert, David H, Neural Computation 8.7 (1996): 1341-1390.]。在实践中，因为样本特征的数量、数据中的噪声以及是否线性可分等各种情况有所不同，所以我们建议至少要比较几种不同学习算法的性能，以选择适合特定问题的最佳模型。

分类器的计算性能以及预测能力，最终在很大程度上取决于可供学习的基础数据。可以把监督机器学习算法训练的五个主要步骤概括如下：

1. 选择特征并收集已标注的训练样本。
2. 选择性能度量指标。
3. 选择分类器并优化算法。
4. 评估模型的性能。
5. 算法调优。

本书所采用的方法是逐步构建机器学习知识，本章将主要聚焦在不同算法的主要概念上面，并回顾诸如特征选择、预处理、性能指标和超参数调优等主题，我们将在本书的后半部分对此进行更为详细的讨论。

# 3.2　了解scikit-learn的第一步——训练感知器

在第2章中，我们学习了感知器规则和Adaline两个机器学习分类算法，并亲手用Python和NumPy实现了代码。现在我们来看一下scikit-learn的API，如前所述，它结合了对用户友好的界面和高度优化的几种分类算法。scikit-learn软件库不仅提供了大量的学习算法，同时也包含了预处理数据、微调和评估模型等许多方便的功能。在第4章和第5章中，我们将对此及其他基本概念进行更详细的讨论。

作为理解scikit-learn软件库的起点，本章将训练类似在第2章中实现的那种感知器。为了简单起见，本书将在以后章节中继续使用已经熟悉的鸢尾花数据集。这么做很方便，因为该数据集简单且常见，经常被用于测试和检验算法，况且我们已经在前面使用scikit-learn软件库的过程中获得了该数据集。下面将用鸢尾花数据集的两个特征来实现可视化。

我们将把150个鸢尾花样本的花瓣长度和宽度存入特征矩阵X，把相应的品种分类标签存入向量y：

In [1]:
from sklearn import datasets
import numpy as np

In [3]:
iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print('Class labels: ', np.unique(y))

Class labels:  [0 1 2]


为了评估经过训练的模型对未知数据处理的效果，我们再进一步将数据集分割成单独的训练数据集和测试数据集：

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

利用scikit-learn库`model_selection`模块的`train_test_split`函数，把X和y数组随机拆分为30%的测试数据集（45个样本）和70%的训练数据集（105个样本）。

请注意，`train_test_split`函数在分割数据集之前已经在内部对训练数据集进行了洗牌，否则，所有分类标签为0和1的样本都会被分到训练数据集，所有分类标签为2的45份样本数据都将被分到测试数据集。通过设置`random_state`参数，我们将为内部的伪随机数生成器提供一个固定的随机种子（`random_state=1`），该生成器用于在分割数据集之前进行洗牌。采用这样固定的`random_state`可以确保结果可重现。

最后，我们通过定义`stratify=y`获得内置的分层支持。这种分层意味着调用`train_test_split`方法可以返回与输入数据集的分类标签相同比例的训练数据集和测试数据集。可以调用NumPy的`bincount`函数统计数组中每个值出现数，以验证数据：

In [5]:
print('Labels counts in y: ', np.bincount(y))

Labels counts in y:  [50 50 50]


In [6]:
print('Labels counts in y_train: ', np.bincount(y_train))

Labels counts in y_train:  [35 35 35]


In [7]:
print('Labels counts in y_test: ', np.bincount(y_test))

Labels counts in y_test:  [15 15 15]
